## 6. VQE로 수소 분자 에너지 

VQE 를 이용해서 분자별 에너지를 계산하였습니다.
- 수소 분자 기저에너지


### 🙏 참고한 자료
- [qiskit - Quantum Kernel Machine Learning](https://qiskit-community.github.io/qiskit-machine-learning/tutorials/03_quantum_kernel.html)

In [ ]:
# !pip install "qiskit[visualization,nature,algorithms]"
# !pip install qiskit-nature qiskit-algorithms
# !pip install --prefer-binary pyscf

In [9]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.problems import ElectronicStructureProblem
from qiskit_nature.second_q.mappers import JordanWignerMapper

from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal
from qiskit_aer.primitives import Estimator as AerEstimator

# 1. 수소 분자 설정
driver = PySCFDriver(atom='H 0 0 0; H 0 0 0.735', basis='sto3g')
problem = driver.run()
hamiltonian = problem.hamiltonian.second_q_op()
# for label, coeff in sorted(hamiltonian.items()):
#     print(f"{coeff:+.8f} * '{label}'")
    
# 2. Second-quantized Hamiltonian 생성
# second_q_ops = problem.second_q_ops()
# main_op = second_q_ops[0]

# 3. 매핑: Jordan-Wigner → Qubit 연산자
mapper = JordanWignerMapper()
qubit_op = mapper.map(hamiltonian)

# 4. Ansatz 및 Optimizer
ansatz = TwoLocal(qubit_op.num_qubits, ['ry', 'rz'], 'cx', reps=1)
optimizer = COBYLA(maxiter=100)
estimator = AerEstimator()  # Qiskit Aer 백엔드 사용

# 5. VQE 실행
vqe_solver = VQE(ansatz=ansatz, optimizer=optimizer, estimator=estimator)
result = vqe_solver.compute_minimum_eigenvalue(qubit_op)

# 6. 결과 출력
print("H₂ 분자의 바닥상태 에너지 (VQE):", result.eigenvalue.real)


H₂ 분자의 바닥상태 에너지 (VQE): -1.8390470307399696


In [8]:
import pandas as pd
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.problems import ElectronicStructureProblem
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal
from qiskit_aer.primitives import Estimator as AerEstimator

# 분자 리스트
molecules = [
    {"name": "H2", "atom": "H 0 0 0; H 0 0 0.735"},
    {"name": "LiH", "atom": "Li 0 0 0; H 0 0 1.6"},
    {"name": "BeH2", "atom": "Be 0 0 0; H 0 0 1.2; H 0 0 -1.2"},
]

results = []

for mol in molecules:
    try:
        driver = PySCFDriver(atom=mol["atom"], basis="sto3g")
        problem = driver.run()
        hamiltonian = problem.hamiltonian.second_q_op()
        second_q_ops = problem.second_q_ops()
        main_op = second_q_ops[0]

        mapper = JordanWignerMapper()
        qubit_op = mapper.map(main_op)

        ansatz = TwoLocal(qubit_op.num_qubits, ['ry', 'rz'], 'cx', reps=1)
        optimizer = COBYLA(maxiter=100)
        estimator = AerEstimator()

        vqe_solver = VQE(ansatz=ansatz, optimizer=optimizer, estimator=estimator)
        result = vqe_solver.compute_minimum_eigenvalue(qubit_op)

        results.append({
            "molecule": mol["name"],
            "num_qubits": qubit_op.num_qubits,
            "energy": result.eigenvalue.real
        })

        print(f"{mol['name']} 계산 완료: Energy = {result.eigenvalue.real:.6f}")

    except Exception as e:
        print(f"{mol['name']} 계산 중 오류 발생: {e}")
        results.append({
            "molecule": mol["name"],
            "num_qubits": "Error",
            "energy": str(e)
        })

# CSV 저장
df = pd.DataFrame(results)
df.to_csv("vqe_molecule_energies.csv", index=False)
print("✅ 결과가 vqe_molecule_energies.csv 파일로 저장되었습니다.")


H2 계산 완료: Energy = -1.561368
LiH 계산 완료: Energy = -7.865172
BeH2 계산 완료: Energy = -14.715826
✅ 결과가 vqe_molecule_energies.csv 파일로 저장되었습니다.
